# Advanced Raster Analysis

## Learning objectives
- Carry out a supervised classification on a SpatRaster
- Construct a raster sieve using the patches() function
- Deal with thematic (categorical maps)

# Introduction to Sentinel-2 data used here

We will carry out a supervised classification using Sentinel 2 data for the Gewata region in Ethiopia. To do this, we use atmospherically corrected Level 2A data acquired on December 27, 2020. These data were downloaded from ESA’s online data hub, a part of the Copernicus European Programme. As it is freely available, Sentinel data has been commonly used next to Landsat data for environmental monitoring.

![Landsat.v.Sentinel-2](../figs/Landsat.v.Sentinel-2.jpg)

# Notebook Setup

In [ ]:
import os
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import rioxarray as rxr
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep

from scipy.stats import pearsonr

In [ ]:
ROOT_DIR = Path(os.getcwd()).parent

In [ ]:
raster_path = ROOT_DIR / 'S2B2A_T36NZP_20201227T075239_20m_gewata_crop.tif' # the path to your imagery (raster)
bands_file_path = ROOT_DIR / "S2B2A_T36NZP_20201227T075239_20m_gewata_bands.txt"

# Data exploration
Download the data to your computer and open your preferred R IDE to the directory of this tutorial.

After downloading the data we begin with visualization. The data consists of all the Sentinel-2 bands at a spatial resolution (or pixel size) of 20 m, meaning that each pixel on the scene corresponds to a ground distance of 20 m by 20 m. We will also make use of training polygons for the land cover classification, which will be introduced later.

In [ ]:
with open(bands_file_path, "r") as f:
    band_names = f.readlines()
band_names = [name.strip() for name in band_names]

gewata = rxr.open_rasterio(raster_path)
gewata

In [ ]:
# Convert DataArray to Dataset
gewata_ds = gewata.to_dataset(dim="band")

# Get the original band names
original_band_names = gewata.band.values

# Rename the data variables
rename_dict = {original_band_names[i]: band_names[i] for i in range(len(original_band_names))}
gewata_ds = gewata_ds.rename(rename_dict)

In [ ]:
# Sanity check, look at the "data variables", if the renaming is successful, the data variables will be the same as the band names
gewata_ds

In [ ]:
# Convert Dataset back to DataArray
gewata = gewata_ds.to_array(dim="band")

# The image is cloud-free, so drop the cloud mask layer
gewata = gewata.drop_sel(band="SCL")

In [ ]:
# Calculate number of rows and columns for subplots
num_bands = len(gewata.band)
num_cols = 3  # Adjust the number of columns as desired
num_rows = (num_bands + num_cols - 1) // num_cols

# Create subplots
fig, axes = plt.subplots(
    num_rows,
    num_cols,
    figsize=(15, 3*num_rows),
    # sharex=True,
    sharey=True
    )

# Flatten axes if needed
axes = axes.flatten()

# Plot histograms for each band
for i, band in enumerate(gewata):
    ax = axes[i]
    ax.hist(band.values.ravel(), bins=50)
    
    # Add a vertical line for the mean
    mean_value = band.mean().values
    ax.axvline(mean_value, color='r', linestyle='--', label=f'Mean: {mean_value:.2f}')
    # ax.legend()
    
    ax.set_title(f'Histogram of {band.band.values}')
    if i % num_cols == 0:
        ax.set_ylabel('Frequency')
    if i >= num_bands - num_cols:
        ax.set_xlabel('Pixel Value')

plt.tight_layout()
plt.show()

In [ ]:
# Calculate summary statistics for all bands
summary_data = []
for band in gewata:
    summary_data.append({
        "Band": band.band.values.item(),
        "Mean": float(band.mean().values),
        "Std Dev": float(band.std().values),
        "Min": float(band.min().values),
        "Max": float(band.max().values)
    })

# Convert summary data to DataFrame
summary_df = pd.DataFrame(summary_data)

# Print the summary DataFrame
print("Summary Statistics for all Bands:")
summary_df

Note that the values of these bands have been rescaled by a factor of 10,000. This is done for file storage considerations. For example, a value of 0.5643 stored as a float takes up more disk space than a value of 5643 stored as an integer. If you prefer reflectance values in their original scale (from 0 to 1), this can easily be done using raster algebra or app(). We will do this later.

In [ ]:
# convert DataSet into pandas DataFrame
gewata_df = gewata_ds.to_dataframe()

# from all the raster value, randomly pick 1000 cells from each bands
sampled_df = gewata_df.sample(n=1000, random_state=42)
sampled_df = sampled_df.drop(columns=['SCL', 'spatial_ref']) # drop unused column for visualization

In [ ]:
# https://stackoverflow.com/questions/63416894/correlation-values-in-pairplot
def reg_coef(x,y,label=None,color=None,**kwargs):
    ax = plt.gca()
    r,p = pearsonr(x,y)
    text_color = plt.cm.RdYlBu((r + 1) / 2)  # Map correlation value to color
    text_size = abs(r) * 20  # Scale text size based on absolute correlation value
    ax.annotate(
        'r = {:.2f}'.format(r),
        xy=(0.5, 0.5),
        xycoords='axes fraction',
        ha='center',
        # color=text_color,
        fontsize=text_size
    )
    ax.set_axis_off()

g = sns.PairGrid(sampled_df)
g.map_diag(sns.histplot)
g.map_lower(sns.histplot)
g.map_upper(reg_coef)

## Question 1: Given what we know about the location of these bands along the EM spectrum, how could these scatterplots be explained?

<details>
<summary>Click to see answer.</summary>

These details <em>remain</em> <strong>hidden</strong> until expanded.

<pre><code>PASTE LOGS HERE</code></pre>

</details>